In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
kf = KFold(n_splits=3)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [2 3 4 5] TEST: [0 1]
TRAIN: [0 1 4 5] TEST: [2 3]
TRAIN: [0 1 2 3] TEST: [4 5]


In [5]:
train = pd.read_csv('../temp/train_temp50000.csv')
test = pd.read_csv('../temp/test_temp50000.csv')

In [6]:
print(train.shape)
train.head()

(50000, 434)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [18]:
card = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']
for f in card:
    print(str(f) + ":" + str(train[f].nunique()))

card1:5446
card2:497
card3:58
card4:4
card5:74
card6:4


In [8]:
# prob特征
target = "isFraud"
dummy_feat = ['card2']
concat = train.drop(target, axis=1).append(test)

for feat in dummy_feat:
    feat_dummies = pd.get_dummies(concat[feat])
    feat_dummies.columns = ['card5_%s_'%feat + '%s'%col for col in feat_dummies.columns]
    feat_dummies['card5'] = concat['card5'].values
    feat_dummies = feat_dummies.groupby('card5').mean()
    feat_dummies['card5'] = feat_dummies.index
    member = member.merge(feat_dummies, on='card5', how='left')

train_temp = train.merge(member, on='card5', how='left')
test_temp = test.merge(member, on='card5', how='left')

train['prob_card5|card2'] = train_temp[[col for col in train_temp.columns if 'card2' in col]].apply(lambda x: \
        x['card5_card2_%d'%x['card2']], axis=1)
test['prob_card5|card2'] = test_temp[[col for col in test_temp.columns if 'card2' in col]].apply(lambda x: \
        x['card5_card2_%d'%x['card2']], axis=1)

NameError: name 'member' is not defined

In [78]:
train['card_str'] = train["card1"].apply(lambda x: str(x)) + "_" + train["card2"].apply(lambda x: str(x)) +\
                "_" + train["card3"].apply(lambda x: str(x)) + "_" + train["card4"].apply(lambda x: str(x)) +\
                "_" + train["card5"].apply(lambda x: str(x)) + "_" + train["card6"].apply(lambda x: str(x))

test['card_str'] = test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
                "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
                "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x))

In [79]:
train['card_count_full'] = train['card_str'].map(pd.concat([train['card_str'], test['card_str']], ignore_index=True).value_counts(dropna=False))

test['card_count_full'] = test['card_str'].map(pd.concat([test['card_str'], test['card_str']], ignore_index=True).value_counts(dropna=False))



In [90]:
train.TransactionAmt.head(100)

0      68.500
1      29.000
2      59.000
3      50.000
4      50.000
5      49.000
6     159.000
7     422.500
8      15.000
9     117.000
10     75.900
11     16.500
12     50.000
13     40.000
14     10.500
15     57.940
16     30.000
17    100.000
18     47.940
19    186.000
20     39.000
21    160.000
22     50.000
23    107.940
24     73.940
25    107.940
26    184.000
27     47.940
28     20.000
29     37.000
       ...   
70    100.000
71     59.000
72      6.766
73    554.000
74     27.800
75     68.500
76     36.940
77    280.000
78    300.000
79     28.700
80     60.000
81    104.940
82    280.000
83    412.000
84    125.700
85     42.600
86     44.500
87     88.940
88    140.000
89    319.000
90     77.800
91    107.000
92    117.000
93     50.000
94    527.000
95     59.000
96    108.500
97     21.500
98    159.000
99     75.000
Name: TransactionAmt, Length: 100, dtype: float64

In [69]:
train['card1_card2_card3'].head()

0      13926_nan_150.0
1     2755_404.0_150.0
2     4663_490.0_150.0
3    18132_567.0_150.0
4     4497_514.0_150.0
Name: card1_card2_card3, dtype: object

In [58]:
train['card6'].unique()

array(['credit', 'debit', nan, 'debit or credit', 'charge card'],
      dtype=object)

In [43]:
train['card4'].unique()

array(['discover', 'mastercard', 'visa', 'american express', nan],
      dtype=object)

In [42]:
train['card3'].nunique()

4

In [64]:
train.loc[train["card6"] == "debit or credit", card + ["isFraud"]].head(100)

,card1,card2,card3,card4,card5,card6,isFraud
16408,4648,555.0,150.0,mastercard,117.0,debit or credit,0
27859,2572,555.0,150.0,mastercard,117.0,debit or credit,0
43216,4648,555.0,150.0,mastercard,117.0,debit or credit,0


In [57]:
train["card3"].nunique()

58

In [65]:
train.loc[train["card6"] == "debit or credit", card + ["isFraud"]].head(100)

,card1,card2,card3,card4,card5,card6,isFraud
16408,4648,555.0,150.0,mastercard,117.0,debit or credit,0
27859,2572,555.0,150.0,mastercard,117.0,debit or credit,0
43216,4648,555.0,150.0,mastercard,117.0,debit or credit,0


0          13926_nan
1         2755_404.0
2         4663_490.0
3        18132_567.0
4         4497_514.0
5         5937_555.0
6        12308_360.0
7        12695_490.0
8         2803_100.0
9        17399_111.0
10       16496_352.0
11        4461_375.0
12        3786_418.0
13       12866_303.0
14       11839_490.0
15        7055_555.0
16        1790_555.0
17       11492_111.0
18        4663_490.0
19        7005_111.0
20        7875_314.0
21       11401_543.0
22        1724_583.0
23        2392_360.0
24       10112_360.0
25       15385_111.0
26       17868_148.0
27       11307_321.0
28        8431_269.0
29       12932_361.0
            ...     
49970     6019_583.0
49971     6019_583.0
49972    12686_215.0
49973    16075_514.0
49974     3343_513.0
49975    15111_310.0
49976    12722_555.0
49977     5627_555.0
49978     4461_375.0
49979     1522_208.0
49980    17947_371.0
49981     6394_225.0
49982    12544_321.0
49983     9069_399.0
49984     3335_545.0
49985     3821_111.0
49986     341